In [ ]:
import string
import re
import numpy as np
import time
import csv
import operator
import os
import xml.etree.ElementTree as etree
from bs4 import BeautifulSoup

**Preprocessing the text**

In [ ]:
def textprocessing(txt):
  txt = txt.translate(str.maketrans('','',string.punctuation))
  txt = re.sub(' \d+',' ',txt)                               
  txt = txt.lower()                                         
  txt = '<s ' + txt + ' /s>'
  txt = re.sub('\s{2,}',' ',txt)
  return txt

**Extracting the sentences from the directories**

In [ ]:
train_path = '/home/harsha/Downloads/dataset'
test_path = '/home/harsha/Downloads/2007'
def getlist(path):
    indexes = []
    stories = []

    for files in os.listdir(path):
        if(len(files) == 26):
            indexes.append(files)
        else:
            stories.append(files)
    return indexes,stories


test_index,test_stories = getlist(test_path)
train_index,train_stories = getlist(train_path)
print("no of train stories")
len(train_stories)

In [ ]:
def getSentences(path,indexes,stories):
    sentences = []
    n = int(len(indexes)/2)

    for i in range(n):
        filepath = os.path.join(path,indexes[i])
        with open(filepath,'r') as f:
                soup = BeautifulSoup(f, 'xml')
                for text in soup.find_all('TEXT'):
                    for line in text.text.splitlines()[7:-5]:
                        if line : 
                            sentences.append(line)
    n=200
    for i in range(n):
        filepath = os.path.join(path, stories[i])
        with open(filepath,'r') as f:
            soup = BeautifulSoup(f, 'xml')
            for text in soup.find_all('TEXT'):
                result = list(filter(lambda x: x, text.text.splitlines()[7:]))
                for line in result:
                    sentences.append(line)
    return sentences

In [ ]:
train_sentences = getSentences(train_path,train_index,train_stories)
test_sentences = getSentences(test_path,test_index,test_stories)

train = list(map(textprocessing,train_sentences))
test = list(map(textprocessing,test_sentences))

train_data = ' '.join(train)
test_data = ' '.join(test)

train_words = list(set(train_data.split()))
test_words = list(set(test_data.split()))

**Unigram model**

In [ ]:
train_unigram = dict()
test_unigram = dict()

train_len = len(train_words)
for word in train_words:
    train_unigram[word] = len(re.findall(' ' + word + ' ', train_data))/train_len

test_len = len(test_words)
for word in test_words:
    test_unigram[word] = len(re.findall(' ' + word + ' ', test_data))/test_len


**Bigram model**

In [ ]:
words = train_data.split()
bigram = dict()
start = time.process_time()         
for i in range(len(words)-1):
    temp = ' ' + words[i] + ' ' + words[i+1] + ' '
    key = words[i] + ' ' + words[i+1]
    try:
        bigram[key] = len(re.findall(temp,train_data))/(train_unigram[words[i]]*train_len)
    except:
        continue
print(time.process_time() - start)

274.52249041000005


In [ ]:
def getBigramSent(current_word):
    sent = current_word
    current_word = current_word.split()
    while(1):
        search = current_word[-1] + ' '
        temp = [ (key,w) for key,w in bigram.items() if key.startswith(search)]
        temp.sort(key=operator.itemgetter(1))
        temp.reverse()
        print(temp[0])
        current_word = temp[0][0].split()
        sent = sent + " " + current_word[-1]
        if('/s>' in current_word):
            return sent

current_word = '<s an'
print(getBigramSent(current_word))


('an official', 0.026717557251908396)
('official said', 0.3389830508474576)
('said today', 0.23192019950124687)
('today /s>', 0.08928571428571429)
<s an official said today /s>


In [ ]:
def BigramTest(sent):
  per_bi = 1
  temp = sent.split()
  N = len(temp)
  for i in range(N-1):
    bigr = temp[i] + ' ' + temp[i+1]
    if(bi in bigram.keys() and bigram[bigr]!=0):
        per_bi = per_bi*bigram[bigr]
    else:
        per_bi = per_bi*test_unigram[temp[i]]*test_unigram[temp[i+1]]
        
    
    return per_bi**(-1/N)


**Trigram model**

In [ ]:
trigram = dict()
start = time.process_time()

for i in range(len(words)-2):
    temp = ' ' + words[i] + ' ' + words[i+1] + ' ' + words[i+2]  + ' '
    key = words[i] + ' ' + words[i+1] + ' ' + words[i+2]
    bigr = words[i] + ' ' + words[i+1]
    try:
        if bigram[bigr]:
            trigram[key] = len(re.findall(temp,train_data))/(bigram[bigr]*train_len)
    except:
        continue
print(time.process_time() - start)
    

240.59085678499997


In [ ]:
def getTriSent(current_word):
    sent = current_word
    current_word = current_word.split()
    while(1):
        search = current_word[-1] + ' '
        temp = [ (key,w) for key,w in trigram.items() if key.startswith(current_word[-1])]
        temp.sort(key=operator.itemgetter(1))
        temp.reverse()
        print(temp[0])
        current_word = temp[0][0].split()
        sent = sent + " " + ' '.join(temp[0][0].split()[1:])
        if('/s>' in current_word):
            return sent
current_word = '<s two buses'
print('\n ' + getTriSent(current_word))

('buses police and', 0.00048344210780759005)
('and destroyed cars', 0.14438804286520024)
('cars the body', 0.0013697526387881718)
('body part /s>', 0.0015309000080573685)

 <s two buses police and destroyed cars the body part /s>


In [ ]:
def TrigramTest(sent):
  per_tri = 1
  temp = sent.split()
  N = len(temp)
  for i in range(N-2):
    tri = temp[i] + ' ' + temp[i+1] + ' ' + temp[i+2]
    try:
        if trigram[tri]:
            per_tri = per_tri*trigram[tri]
        else:
            try:  
                per_tri = prer_tri*test_unigram[temp[i]]*test_unigram[temp[i+1]]*test_unigram[temp[i+2]]
            except:
                continue
    except:
        continue

  return per_tri**(-1/N)

In [ ]:
p_bigram = 1
p_trigram = 1
for lines in test:
    p_bigram = p_bigram*BigramTest(lines)
    p_trigram = p_trigram*TrigramTest(lines)

**Perplexities**

In [ ]:
print("perplexity for bigram")
p_bigram

perplexity for bigram


2.790398554129296e+286

In [ ]:
print("preplexity for trigram")
p_trigram

preplexity for trigram


inf

**Using inbuilt nltk**

In [ ]:
def textprocessing2(txt):
  txt = txt.translate(str.maketrans('','',string.punctuation))
  txt = txt.lower()                                           #converts to lower case
  txt = re.sub(' \d+',' ',txt)                                #deleting the numbers
  txt = re.sub('\s{2,}',' ',txt)
  return txt

In [ ]:
train = list(map(textprocessing2,sentences_train))
train_temp =[]
for line in train:
    train_temp.append(line.split())

test = list(map(textprocessing2,sentences_test))
test_temp =[]
for line in test:
    test_temp.append(line.split())

In [ ]:
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.4

Requirement already up-to-date: pip in ./ENTER/lib/python3.7/site-packages (20.0.2)
     |████████████████████████████████| 151 kB 584 kB/s eta 0:00:01
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=77454 sha256=658921fea65235e53c166ddea0ca34679aeced8daa19f4d3cd4c4205d0b85318
  Stored in directory: /home/harsha/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
Successfully built dill
     |████████████████████████████████| 1.4 MB 869 kB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1435855 sha256=b2a035a43b395c3f3e0203a879f9c383f1f99ad7086eaa48a17e983249faf306
  Stored in directory: /home/harsha/.cache/pip/wheels/13/b8/81/2349be11dd144dc7b68ab983b58cd2fae353cdc50bbdeb09d0
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.3
    Uninstalling nltk-3.3:
      Successfully uninstalled nltk-3.3


In [ ]:
import nltk
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(2, train_temp)

In [ ]:
from nltk.lm import MLE
lm1 = MLE(1)
lm2 = MLE(2)
lm3 = MLE(3)
lm1.fit(train,vocab)
lm2.fit(train,vocab)
lm3.fit(train,vocab)

**Perplexities of uni,bi,trigram**

In [ ]:
print("preplexity for unigram")
lm1.perplexity(test_temp[:1000])

preplexity for unigram


inf

In [ ]:
print("preplexity for bigram")
lm1.perplexity(test_temp[:1000])

preplexity for bigram


inf

In [ ]:
print("preplexity for trigram")
lm1.perplexity(test_temp[:1000])

preplexity for trigram


inf

**Sentence generation**

In [ ]:
print(lm1.generate(4, text_seed=['islamabad'], random_seed=2))
print(lm1.generate(10,random_seed=2))

['yesterday', 'were', 'up', 'under']
['ways', 'to', 'use', 'trade', 'union', 'were', 'up', 'under', 'way', 'today']


In [ ]:
print(lm1.generate(4, text_seed=['visit'], random_seed=2))
print(lm1.generate(10,random_seed=2))

['to', 'use', 'trade', 'union']
['ways', 'to', 'use', 'trade', 'union', 'were', 'up', 'under', 'way', 'today']
